STAT 453: Deep Learning (Spring 2021)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)  

Course website: http://pages.stat.wisc.edu/~sraschka/teaching/stat453-ss2021/  
GitHub repository: https://github.com/rasbt/stat453-deep-learning-ss21

---

# RNN Classifier with LSTM Trained on Own Dataset (IMDB)

Example notebook showing how to use an own CSV text dataset for training a simple RNN for sentiment classification (here: a binary classification problem with two labels, positive and negative) using LSTM (Long Short Term Memory) cells.

In [1]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd

## General Settings

In [23]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 1
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

Check that the dataset looks okay:

In [5]:
df = pd.read_csv('C:\\Users\\Rogonfu\\Downloads\\movie\\movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [6]:
del df

## Prepare Dataset with Torchtext

In [ ]:
# !conda install spacy

Download English vocabulary via:
    
- `python -m spacy download en_core_web_sm`

Define the Label and Text field formatters:

In [7]:
### Defining the feature processing

TEXT = torchtext.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.data.LabelField(dtype=torch.long)

Process the dataset:

In [8]:
fields = [('review', TEXT), ('sentiment', LABEL)]

dataset = torchtext.data.TabularDataset(
    path='C:\\Users\\Rogonfu\\Downloads\\movie\\movie_data.csv', format='csv',
    skip_header=True, fields=fields)

In [ ]:
LABEL

## Split Dataset into Train/Validation/Test

Split the dataset into training, validation, and test partitions:

In [9]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 40000
Num Test: 10000


In [ ]:
"""train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')"""

In [10]:
print(vars(train_data.examples[0]))

{'review': ['Pros', ':', 'Nothing', '<', 'br', '/><br', '/>Cons', ':', 'Everything', '<', 'br', '/><br', '/>Plot', 'summary', ':', 'A', 'female', 'reporter', 'runs', 'into', 'a', 'hitchhiker', 'that', 'tells', 'her', 'stories', 'about', 'the', 'deaths', 'of', 'people', 'that', 'were', 'killed', 'by', 'zombies.<br', '/><br', '/>Review', ':', 'Never', 'in', 'my', 'life', 'have', 'I', 'come', 'across', 'a', 'movie', 'as', 'bad', 'The', 'Zombie', 'Chronicles', '.', 'Filmed', 'on', 'a', 'budget', 'of', 'what', 'looks', 'to', 'be', 'about', '20', 'bucks', ',', 'TZC', 'is', 'a', 'completely', 'horrible', 'horror', 'movie', 'that', 'relies', 'on', 'lame', ',', 'forgetable', 'actors', 'whom', 'could', "n't", 'act', 'to', 'save', 'their', 'lives', 'and', 'gore', 'that', "'s", 'more', 'gross', 'than', 'frightening', '.', 'How', 'does', 'a', 'movie', 'like', 'this', 'even', 'get', 'made', '?', 'Simply', 'put', ',', 'avoid', 'TZC', 'like', 'a', 'sexually', '-', 'transmitted', 'disease.<br', '/><br'

## Build Vocabulary

Build the vocabulary based on the top "VOCABULARY_SIZE" words:

In [11]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


- 20,002 not 20,000 because of the `<unk>` and `<pad>` tokens
- PyTorch RNNs can deal with arbitrary lengths due to dynamic graphs, but padding is necessary for padding sequences to the same length in a given minibatch so we can store those in an array

**Look at most common words:**

In [25]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 459609), (',', 435431), ('.', 374990), ('and', 247489), ('a', 247209), ('of', 228621), ('to', 211349), ('is', 171445), ('in', 139456), ('I', 124259), ('it', 121568), ('that', 110274), ('"', 101770), ("'s", 97508), ('this', 96060), ('-', 83922), ('/><br', 80673), ('was', 79048), ('as', 67990), ('movie', 67735)]


**Tokens corresponding to the first 10 indices (0, 1, ..., 9):**

In [26]:
print(TEXT.vocab.itos[:10]) # itos = integer-to-string

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


**Converting a string to an integer:**

In [ ]:
print(TEXT.vocab.stoi['the']) # stoi = string-to-integer

**Class labels:**

In [ ]:
print(LABEL.vocab.stoi)

**Class label count:**

In [12]:
LABEL.vocab.freqs

Counter({'0': 20025, '1': 19975})

## Define Data Loaders

In [13]:
train_loader, test_loader = \
    torchtext.data.BucketIterator.splits(
        (train_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.review),
         device=DEVICE
    )

Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

In [14]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.review.size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
"""print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.review.size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break"""
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.review.size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break

Train
Text matrix size: torch.Size([1126, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([52, 128])
Target vector size: torch.Size([128])


## Model

In [2]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

NameError: name 'torch' is not defined

In [16]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

## Training

In [17]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [1]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.review.to(DEVICE)
        labels = batch_data.sentiment.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: 'f'{compute_accuracy(model, train_loader, DEVICE):.2f}%')
        
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

NameError: name 'time' is not defined

In [1]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][0].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


NameError: name 'model' is not defined

In [30]:
print('Probability negative:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability negative:


0.4851503074169159